In [6]:
import numpy as np
import collections
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [7]:
# 5 authors
 
# A<-B->C<-D
#    ^  ^
#    |  |
#    E--

# simple example, one paper only per link, one author per paper

In [9]:
#returns id of author chosen uniformly randomly
def randStartPoint():
    idx = np.random.randint(len(author_to_score))
    return list(author_to_score.keys())[idx]   #perhaps sorting issues here? check later

def getScore(cur_author, proposed_author):
    papers = author_to_author_and_papers[cur_author][proposed_author]
    score = 0
    for paper in papers:
        paper_authors = paper_to_authors[paper]
        avg_author_score = np.mean([author_to_score[auth] for auth in paper_authors])
        score += avg_author_score * paper_to_if[paper]
    
    return score

In [37]:
#testing randStartPoint
author_to_start = dict(A1=0, A2=0, A3=0, A4=0, A5=0)
for i in range(1000000):
    author_to_start[randStartPoint()] += 1
print(author_to_start)

{'A1': 199735, 'A2': 199928, 'A3': 199475, 'A4': 200791, 'A5': 200071}


In [84]:
#to do: sum up multiple papers btwn a and b

#update author_to_score every iteration
def graphtraversal(num_trials, num_walks_per_trial):
    
    output = np.zeros((num_trials, numAuthors))   #matrix of all results. sorted by author id's.
    
    for i in range(num_trials):
        print()
        print("trial number ", str(i))
    
        #select uniform start point
        cur_author = randStartPoint()
        
        print("start at ", cur_author)
        
        for num_walk in range(num_walks_per_trial):
            #calculate next author to go to (propose next state) according to dist. 1/d for current node
            if len(author_to_author_and_papers[cur_author]) == 0: #current author is a dead end
                cur_author = randStartPoint()
                print("start again at ", cur_author)
            else:
                
                idx = np.random.randint(len(author_to_author_and_papers[cur_author]))
                proposed_author = list(author_to_author_and_papers[cur_author].keys())[idx]

                print("proposed author: ", proposed_author)

                #accept or not? according to metropolis hastings 
                pi_y = author_to_score[proposed_author]  #pi(x) = score(x)
                pi_x = author_to_score[cur_author]
                if len(author_to_author_and_papers[proposed_author]) == 0:  #if proposed author is a sink
                    f_y = 1
                else:
                    f_y = 1 / len(author_to_author_and_papers[proposed_author]) #f(x) = 1/d where d is degree of node x (number of outgoing edges)
                if len(author_to_author_and_papers[cur_author]) == 0:
                    f_x = 1
                else:
                    f_x = 1 / len(author_to_author_and_papers[cur_author]) #maybe float errors here
                
                print("pi_y: ", str(pi_y))
                print("pi_x: ", str(pi_x))
                print("f_y: ", str(f_y))
                print("f_x: ", str(f_x))
                print("calculated: ", (pi_y * f_y) / (pi_x * f_x))
                p_accept = min(1, (pi_y * f_y) / (pi_x * f_x))

                print("p accept", str(p_accept))

                if np.random.random() < p_accept:
                    #if accept, update proposed_author's score.  Then move to proposed_author
                    author_to_score[proposed_author] += getScore(cur_author, proposed_author)
                    cur_author = proposed_author
                    
                    print("printing all scores before norm...")
                    for author, score in zip(author_to_score.keys(), author_to_score.values()):
                        print("author: ", author)
                        print("score: ", score)

                    #renormalize everyone author's score
                    norm_score = sum(author_to_score.values())
#                     norm_score = max(author_to_score.values())

                    print("printing all scores...")
                    for author in author_to_score:
                        author_to_score[author] = author_to_score[author] / norm_score
                        print("author: ", author)
                        print("score: ", author_to_score[author])

                else:  #if not accept, randomly start again
                    cur_author = randStartPoint()
                    print("start again at ", cur_author)
        
        #add trial results to output matrix
        sum_score = sum(author_to_score.values())
        output[i] = np.array(list(author_to_score.values()))  #hopefully sorted? check here too
                                                        #don't think this numpy notation works. tweak later lol
     
    avg_output = np.mean(output, axis=0) #columnwise avg for each author
    return output, avg_output

In [85]:
numAuthors = 5
startScore = 1
author_to_score = collections.OrderedDict(A1=startScore, A2=startScore, A3=startScore, A4=startScore, A5=startScore)

paper_to_if = dict(P1=5.680, P2=5.680, P3=5.680, P4=5.680, P5=5.680)

#paper id -> list of author id's
paper_to_authors = dict(P1=['A1'], P2=['A2'], P3=['A3'], P4=['A4'], P5=['A5'])

#author a -> (author b that a cited -> list of papers a cited b in)
author_to_author_and_papers = dict( A1=dict(),
                                    A2=dict(A1=['P1'], A3=['P3']),
                                    A3=dict(),
                                    A4=dict(A3=['P3']),
                                    A5=dict(A3=['P3'], A2=['P2']))

#Run MCMC for num_trials, each trial walking num_walks_per_trial steps
output, avg_output = graphtraversal(5, 10)


trial number  0
start at  A3
start again at  A2
proposed author:  A1
pi_y:  1
pi_x:  1
f_y:  1
f_x:  0.5
calculated:  2.0
p accept 1
printing all scores before norm...
author:  A1
score:  6.68
author:  A2
score:  1
author:  A3
score:  1
author:  A4
score:  1
author:  A5
score:  1
printing all scores...
author:  A1
score:  0.6254681647940075
author:  A2
score:  0.09363295880149813
author:  A3
score:  0.09363295880149813
author:  A4
score:  0.09363295880149813
author:  A5
score:  0.09363295880149813
start again at  A3
start again at  A2
proposed author:  A3
pi_y:  0.09363295880149813
pi_x:  0.09363295880149813
f_y:  1
f_x:  0.5
calculated:  2.0
p accept 1
printing all scores before norm...
author:  A1
score:  0.6254681647940075
author:  A2
score:  0.09363295880149813
author:  A3
score:  0.6254681647940075
author:  A4
score:  0.09363295880149813
author:  A5
score:  0.09363295880149813
printing all scores...
author:  A1
score:  0.4083129584352078
author:  A2
score:  0.06112469437652812
au

In [56]:
avg_output

NameError: name 'avg_output' is not defined